<a href="https://colab.research.google.com/github/zwj93/test/blob/master/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install pyspark

     |████████████████████████████████| 215.6MB 125kB/s 
     |████████████████████████████████| 204kB 34.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


In [0]:
import os       #importing os to set environment variable
def install_java():
    #install openjdk
    !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
    #set environment variable
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    !java -version       #check java version
install_java()

openjdk version "11.0.3" 2019-04-16
OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1)
OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)


In [0]:
import re, os.path, time
from pyspark import SparkContext
import string
from ast import literal_eval

In [0]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import *
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MulticlassMetrics

# create the spark context
sc = SparkContext()

In [0]:
import os
print(os.listdir())

['.config', 'drive', 'sample_data']


In [0]:
start_time = time.time()
PUNCTUATION = set(string.punctuation)

def tokenize(text):
    stemmeren = PorterStemmer()
    lowercase = [tkn.lower() for tkn in text]
    rm_pun = []
    for i in lowercase:
        i = i.replace('\\n', '')
        punrm = ''.join([charc for charc in i if not charc in PUNCTUATION])
        rm_pun.append(punrm)
    
    stop_rm = [wd for wd in rm_pun if not wd in STOPWORDSEN]
    stem = [stemmeren.stem(wd) for wd in stop_rm]
    return [j for j in stem if j]

#total 59 crash
filePath = "drive/My Drive/Colab Notebooks/crash.txt"
crashData = sc.textFile(filePath).map(lambda r:str(r)).map(lambda line: line.split(" "))
crashDataRDD = crashData.map(lambda t:(0,t))

#total 219 nearCrash
filePath = "drive/My Drive/Colab Notebooks/nearCrash.txt"
nearCrashData = sc.textFile(filePath).map(lambda r:str(r)).map(lambda line: line.split(" "))
nearCrashRDD = nearCrashData.map(lambda t:(1,t))

#total 9 otherEvent
filePath = "drive/My Drive/Colab Notebooks/otherType.txt"
otherEventData = sc.textFile(filePath).map(lambda r:str(r)).map(lambda line: line.split(" "))
otherEventDataRDD = otherEventData.map(lambda t:(2,t))

experimentData = crashDataRDD + nearCrashRDD + otherEventDataRDD

# load stop words list
stopWordsPath = "drive/My Drive/Colab Notebooks/stop-word-list.txt"
stopWordsSet  = sc.textFile(stopWordsPath).collect()
STOPWORDSEN   = set(stopWordsSet)

tokens_data = experimentData.map(lambda x: (x[0], tokenize(x[1])))
tokens_data.saveAsTextFile("tokens_data/")

In [0]:
cleanedData = sc.textFile("tokens_data/").map(lambda r:literal_eval(str(r)))

TypeError: ignored

In [0]:
# feature extraction
distictNumberOfWords = cleanedData.flatMap(lambda x:x[1]).distinct().count()
htf = HashingTF(distictNumberOfWords+200)
hashedData = cleanedData.map(lambda x: LabeledPoint(x[0], htf.transform(x[1])))
hashedData.saveAsTextFile("hashed_data/")

<class 'pyspark.mllib.feature.HashingTF'>


In [0]:
pip install pyspark.mllib

  ERROR: Could not find a version that satisfies the requirement pyspark.mllib (from versions: none)
ERROR: No matching distribution found for pyspark.mllib


In [0]:
from pyspark.mllib.linalg import Vectors
freqs = [Vectors.dense([0.0, 1.0, 0.0, 3.0]), Vectors.dense([1.0, 4.0, 2.0, 3.0]), Vectors.dense([0.0, 0.0, 2.0, 3.0])]

In [0]:
freqs
data = sc.parallelize(freqs)
model = IDF().fit(data)

[DenseVector([0.0, 1.0, 0.0, 3.0]),
 DenseVector([1.0, 4.0, 2.0, 3.0]),
 DenseVector([0.0, 0.0, 2.0, 3.0])]

In [0]:
print(type(model))

<class 'pyspark.mllib.feature.IDFModel'>


In [0]:
model.transform(Vectors.dense([2.0, 3.0]))

DenseVector([1.3863, 0.863])

In [0]:
import math
math.log(4 / 3) * 3

0.8630462173553426